In [1]:
import numpy as np
import pandas as pd
from more_itertools import chunked, ichunked
from pmf import ProbabilisticMatrixFactorization

In [2]:
file_name = '../archive/combined_data'
movies = []
user_ratings = {}
current_movie = None

for i in range(1, 5):
    with open(f'{file_name}_{i}.txt', 'r') as current_file:
        print(f'Processing combined_data_{i}.txt...')
        j = 0
        for line in current_file.readlines():
            elems = line.split(',')
            j += 1
            if j % 5_000_000 == 0:
                print(f'\t{j} lines processed...')
            if len(elems) == 1:
                movie = elems[0].strip(':\n')
                movies.append(movie)
                current_movie = movie
            elif len(elems) == 3:
                user_id, user_rating = elems[0], int(elems[1])
                ratings = user_ratings.get(user_id, {})
                ratings[current_movie] = user_rating
                user_ratings[user_id] = ratings
        print(f'\t{j} total lines processed.')
#print(movies)
#print(user_ratings)

Processing combined_data_1.txt...
	5000000 lines processed...
	10000000 lines processed...
	15000000 lines processed...
	20000000 lines processed...
	24058263 total lines processed.
Processing combined_data_2.txt...
	5000000 lines processed...
	10000000 lines processed...
	15000000 lines processed...
	20000000 lines processed...
	25000000 lines processed...
	26982302 total lines processed.
Processing combined_data_3.txt...
	5000000 lines processed...
	10000000 lines processed...
	15000000 lines processed...
	20000000 lines processed...
	22605786 total lines processed.
Processing combined_data_4.txt...
	5000000 lines processed...
	10000000 lines processed...
	15000000 lines processed...
	20000000 lines processed...
	25000000 lines processed...
	26851926 total lines processed.


In [3]:
N = len(list(user_ratings.keys()))
print(f'Number of total users: {N}')

Number of total users: 480189


In [4]:
batch_size = 5_000
j = 1
for chunk in chunked(user_ratings.keys(), batch_size):
    print(f'Processing batch number {j} -> {len(chunk)} users')
    data = []
    for i, key in enumerate(chunk):
        data.append([])
        for movie in movies:
            if movie in user_ratings[key].keys():
                data[-1].append(user_ratings[key][movie])
            else:
                data[-1].append(0)
        if (i+1) % 2_500 == 0:
            print(f'\t{i+1} users processed...')
    pmf = ProbabilisticMatrixFactorization(D=10, sigma=0.1, sigma_u=0.1, sigma_v=0.1, max_epochs=10)
    print(f'Fitting new values with PMF model...')
    pmf.fit(np.array(data))
    print(f'Batch {j} processed!')
    j += 1

Processing batch number 1 -> 5000 users
	2500 users processed...
	5000 users processed...
Fitting new values with PMF model...
	Epoch-1...
	Epoch-2...
	Epoch-3...


c:\Users\kmart\Desktop\Apuntes\MasterPython\TFM\PMF_TFM\pmf.py:72: RuntimeWarning: overflow encountered in multiply
  aux_2 = aux_1 * self.U.T


	Epoch-4...


c:\Users\kmart\Desktop\Apuntes\MasterPython\TFM\PMF_TFM\pmf.py:56: RuntimeWarning: invalid value encountered in multiply
  aux_1 = I_ij * (self.R[i, :] - np.dot(self.U[i, :], self.V))


	Epoch-5...


KeyboardInterrupt: 

In [ ]:
#data = np.array([[user_ratings[key][movie] if movie in user_ratings[key].keys() else 0 for movie in movies] for key in user_ratings.keys()], dtype=np.int16)
# data = []
# for i, key in enumerate(user_ratings.keys()):
#     data.append([])
#     for movie in movies:
#         if movie in user_ratings[key].keys():
#             data[-1].append(user_ratings[key][movie])
#         else:
#             data[-1].append(0)
#     if (i+1) % 10_000 == 0:
#         print(f'{i} users processed...')

# data = np.array(data)
# df = pd.DataFrame(data=data, index=list(user_ratings.keys()), columns=movies)
# df.head()